In [1]:
filepath = 'Day11Input.txt'

# Problem 1

In [2]:
class Monkey:
    def __init__(self, items, operation, operation_value, test_divisor, true_test_target, false_test_target):
        self.items = items
        self.operation = operation
        try:
            self.operation_value = int(operation_value)
        except ValueError:
            self.operation_value = operation_value
        self.test_divisor = test_divisor
        self.true_test_target = true_test_target
        self.false_test_target = false_test_target
        
        self.inspection_count = 0
    
    def process(self, monkeys, relief=True, debug=False):
        self.inspection_count += len(self.items)
        
        while len(self.items) > 0:
            item = self.items.pop(0)
            
            # Inspect
            if debug:
                print(f'  Monkey inspects an item with a worry level of {item}')
            op_val = self.operation_value if type(self.operation_value) == int else int(item)
            if self.operation == '*':
                if debug:
                    print(f'    Worry level is multiplied by {op_val} to {item * op_val}')
                item = item * op_val
            elif self.operation == '+':
                if debug:
                    print(f'    Worry level increases by {op_val} to {item + op_val}')
                item = item + op_val
            
            # Bored, relief
            if relief:
                if debug:
                    print(f'    Monkey gets bored with item. Worry level is divided by 3 to {item // 3}')
                item = item // 3
            
            # Test
            if item % self.test_divisor == 0:
                if debug:
                    print(f'    Current worry level {item} is divisible by {self.test_divisor}\n    Item with worry level {item} is thrown to monkey {self.true_test_target}', flush=True)
                monkeys[self.true_test_target].items.append(item)
            else:
                if debug:
                    print(f'    Current worry level {item} is not divisible by {self.test_divisor}\n    Item with worry level {item} is thrown to monkey {self.false_test_target}', flush=True)
                monkeys[self.false_test_target].items.append(item)
            
            if debug:
                input()
        
    
    def __str__(self):
        s = f'  Items: {self.items}\n'
        s += f'  Operation: new = old {self.operation} {self.operation_value}\n'
        s += f'  Test: divisible by {self.test_divisor}\n'
        s += f'    If true: throw to monkey {self.true_test_target}\n'
        s += f'    If false: throw to monkey {self.false_test_target}\n'
        return s

In [3]:
monkeys = []
with open(filepath) as f:
    lines = f.readlines()
    
    for i in range(0, len(lines), 7):
        items = [int(x) for x in lines[i + 1].split(': ')[1].split(', ')]
        operation = '*' if '*' in lines[i + 2] else '+'
        operation_value = lines[i + 2].split(operation)[1].strip()
        test_divisor = int(lines[i + 3].split(' ')[-1])
        true_test_target = int(lines[i + 4].split(' ')[-1])
        false_test_target = int(lines[i + 5].split(' ')[-1])
        
        monkeys.append(
            Monkey(
                items,
                operation,
                operation_value,
                test_divisor,
                true_test_target,
                false_test_target
            )
        )

In [4]:
for _ in range(20):
    for m, monkey in enumerate(monkeys):
        # print(f'Monkey {m}:')
        monkey.process(monkeys)
        
for m, monkey in enumerate(monkeys):
    print(f'Monkey {m} inspected {monkey.inspection_count} items')

Monkey 0 inspected 313 items
Monkey 1 inspected 338 items
Monkey 2 inspected 319 items
Monkey 3 inspected 314 items
Monkey 4 inspected 30 items
Monkey 5 inspected 19 items
Monkey 6 inspected 38 items
Monkey 7 inspected 101 items


In [5]:
monkey_inspections = [monkey.inspection_count for monkey in monkeys]
monkey_inspections.sort()
print(monkey_inspections[-1] * monkey_inspections[-2])

107822


# Problem 2

In [6]:
import numpy as np

# Get dimensions of items array
n_items, n_monkeys = 0, 0
for i in range(0, len(lines), 7):
    n_items += len([int(x) for x in lines[i + 1].split(': ')[1].split(', ')])
    n_monkeys += 1

# Create and fill items array with initial item values
items = np.zeros((n_monkeys, n_items), dtype=np.uint64)
monkey_index, item_index = 0, 0
for i in range(0, len(lines), 7):
    monkey_items = [int(x) for x in lines[i + 1].split(': ')[1].split(', ')]
    for item in monkey_items:
        items[monkey_index][item_index] = item
        item_index += 1
    monkey_index += 1

In [7]:
# Create lists for operatations, divisors, targets, and inspection counts
operations = []
divisors = []
targets = []
inspection_counts = [0] * n_monkeys

for i in range(0, len(lines), 7):
    operation = '*' if '*' in lines[i + 2] else '+'
    operation_value = lines[i + 2].split(operation)[1].strip()
    test_divisor = int(lines[i + 3].split(' ')[-1])
    true_test_target = int(lines[i + 4].split(' ')[-1])
    false_test_target = int(lines[i + 5].split(' ')[-1])
    
    operations.append((operation, operation_value))
    divisors.append(test_divisor)
    targets.append((true_test_target, false_test_target))

In [8]:
from math import lcm

scaler = lcm(*divisors)

In [9]:
for round_ in range(10000):
    for monkey_index, monkey in enumerate(items):
        for item_index, item in enumerate(monkey):
            if item != 0:
                # Inspect
                inspection_counts[monkey_index] += 1
                try:
                    op_val = int(operations[monkey_index][1])
                except ValueError:
                    op_val = item
                    
                if operations[monkey_index][0] == '*':
                    item = int(item * op_val)
                elif operations[monkey_index][0] == '+':
                    item = int(item + op_val)
                
                # This keeps everything from getting too big
                item = int(item % scaler)
                
                # Test
                if item % divisors[monkey_index] == 0:
                    items[targets[monkey_index][0]][item_index] = item
                else:
                    items[targets[monkey_index][1]][item_index] = item
                items[monkey_index][item_index] = 0
        
for i in range(len(inspection_counts)):
    print(f'Monkey {i} inspected {inspection_counts[i]} items')
print(inspection_counts)

Monkey 0 inspected 25581 items
Monkey 1 inspected 33619 items
Monkey 2 inspected 12275 items
Monkey 3 inspected 12262 items
Monkey 4 inspected 161071 items
Monkey 5 inspected 147894 items
Monkey 6 inspected 161083 items
Monkey 7 inspected 169274 items
[25581, 33619, 12275, 12262, 161071, 147894, 161083, 169274]


In [10]:
inspection_counts.sort()
print(inspection_counts[-1] * inspection_counts[-2])

27267163742
